In [1]:
import pandas as pd
import numpy as np
import json
import os

In [2]:
drugs = set()

In [6]:
indir = "/scratch/jhm3ab/antibiotic_bii/prescriptions_data/"
for yeardir in os.listdir(indir):
    d = os.path.join(indir, yeardir)
    for filename in os.listdir(d):
        fin = os.path.join(d, filename)
        f = open(fin, 'r')
        new = json.load(f)
        drugs.update(new.keys())
        f.close()

In [7]:
len(drugs)

104212

In [11]:
f = open('alldrugsndc.txt', 'w')
for ndc in drugs:
    f.write(ndc + '\n')

In [12]:
ndcfile = "/scratch/jhm3ab/antibiotic_bii/ndclist/product.txt"
ndcdata = pd.read_csv(ndcfile, sep = '\t', encoding='unicode_escape')

In [13]:
classes = ['Aminoglycoside', 
           'Penicillin',
 'Polymyxin',
 'Lipopeptide',
 'Glycopeptide',
 'Carbapenem',
 'Cephalosporin',
 'Monobactam',
 'Trimethoprim',
 'Chloramphenicol',
 'Macrolide',
 'Oxazolidinone',
 'Quinolone',
 'Tetracycline',
 'Glycylcycline']

In [14]:
len(classes)

15

In [15]:
pharm=ndcdata['PHARM_CLASSES'].unique().tolist()

In [16]:
search = []
failed = []
for c in classes:
    found = False
    for p in pharm:
        p = str(p)
        if c.lower() in p.lower():
            search.append(p)
            found = True
    if not found:
        failed.append(c)

In [17]:
failed

['Trimethoprim', 'Chloramphenicol', 'Glycylcycline']

In [18]:
filtered = ndcdata.loc[ndcdata['PHARM_CLASSES'].isin(search)]
ndcfilter = filtered['PRODUCTNDC'].unique().tolist()
ndc = [n.replace('-','') for n in ndcfilter]

In [20]:
found = []
for key in drugs:
    for n in ndc:
        if n in key:
            found.append(key)

In [22]:
len(found)

1175

In [24]:
f = open('antibiotics_ndc.txt', 'w')
for ndc in found:
    f.write(ndc + '\n')

In [26]:
data = {}

In [27]:
indir = "/scratch/jhm3ab/antibiotic_bii/prescriptions_data/"
for yeardir in os.listdir(indir):
    d = os.path.join(indir, yeardir)
    for filename in os.listdir(d):
        fin = os.path.join(d, filename)
        f = open(fin, 'r')
        new = json.load(f)
        for key in new:
            if key in found:
                frame = pd.DataFrame.from_dict(new[key])
                if key in data:
                    data[key] = pd.concat([data[key], frame], ignore_index = True)
                else:
                    data[key] = frame
        f.close()

In [33]:
for key in data:
    data[key] = data[key].to_dict()
with open('filtered_antibiotics/all_antibiotics', "w") as f:
    json.dump(data, f)

In [ ]:
for c in classes:
    classdata = {}
    search = []
    for p in pharm:
        p = str(p)
        if c.lower() in p.lower():
            search.append(p)
    filtered = ndcdata.loc[ndcdata['PHARM_CLASSES'].isin(search)]
    ndcfilter = filtered['PRODUCTNDC'].unique().tolist()
    ndc = [n.replace('-','') for n in ndcfilter]
    for key in data:
        for n in ndc:
            if n in key:
                classdata[key] = data[key]
    if classdata:
        with open('filtered_antibiotics/' + c + '.json', "w") as f:
            json.dump(classdata, f)